In [2]:
import random
import numpy as np
import pandas as pd

def get_manip(row):
    cond = row["all_cond"]
    if "dom" in cond:
        return "dominant"
    else:
        return "submissive"
    
def get_condition(row):
    cond = row["all_cond"]
    cond = cond.replace("_dom", "")
    cond = cond.replace("_sub", "")
    return cond
        
def clear_non_conflict_manip(row):
    trial_cond = row["trial_cond"]
    manip = row["manip"]

    if trial_cond == "NCE" or trial_cond == "NCH":
        manip = "no_manip"

    return manip


def get_n_roundConsecutives(df): 
    players = ["1", "2", "3", "4", "5", "6"]
    df["other_player_str"] = df["other_participant"].astype(str)
    df["participant_str"] = df["participant"].astype(str)

    distance_dict = {}

    for player in players:
        df_player     = df[df["participant_str"] == player]
        other_players = [p for p in players if p != player]

        distance_dict[player] = 0
        for other in other_players:
            
            last_seen = 0
            for index,row in df_player.iterrows():
                if other == row["other_player_str"]:
                    distance  = row["round_nb"] - last_seen
                    last_seen = row["round_nb"]
                    if distance == 1:
                        distance_dict[f"{player}"] += 1
    return distance_dict


def generate_numbers():
    # Generate a sorted list of 4 unique numbers between 1 and 30
    numbers = sorted(random.sample(range(1, 31), 4))
    
    # Check if the difference between consecutive numbers is at least 2
    for i in range(1, len(numbers)):
        if numbers[i] - numbers[i-1] < 2:
            # If the condition is not met, regenerate the numbers
            return generate_numbers()
    
    return numbers

In [71]:

number = 1
while number <= 20:
    extra_conflict = [
                ("EC_dom", "EC_sub"),
                ("EC_sub", "EC_dom"),
                ("EC_dom", "EC_sub"),
                ("EC_sub", "EC_dom")
                ]
    round_count1 = 0

    extra_rounds = [[[1, 2], [4, 3], [6, 5]], #1
            [[1, 3], [5, 2], [6, 4]],   #2
            [[1, 4], [6, 2], [5, 3]],   #3
            [[1, 5], [4, 2], [6, 3]]]

    conditions = [("ENC_dom", "ENC_sub"),
                ("ENC_sub", "ENC_dom"),
                ("HNC_dom",  "HNC_sub"),
                ("HNC_sub",  "HNC_dom"),
                ("EC_dom", "EC_sub"),
                ("EC_sub", "EC_dom")
                ]
    random.shuffle(conditions)

    rounds = [[[1, 2], [4, 3], [6, 5]], #1
            [[1, 3], [5, 2], [4, 6]],   #2
            [[1, 4], [6, 2], [5, 3]],   #3
            [[1, 5], [4, 2], [3, 6]],   #4
            [[1, 6], [3, 2], [4, 5]],   #5
            ]


    df_extra = pd.DataFrame()

    for (cond_p1, cond_p2) in extra_conflict:
        for (p1, p2) in extra_rounds[round_count1]:
            aux_df = pd.DataFrame()
            aux_df["participant"] = [p1, p2]
            aux_df["other_participant"] = [p2, p1]
            aux_df["all_cond"] = [cond_p1, cond_p2]
            aux_df["extra_trials"] = True

            df_extra = pd.concat([df_extra, aux_df])
        round_count1 +=1
        if round_count1 > 4:
            round_count1=0

    df_extra["manip"]          = df_extra.apply(get_manip, axis=1)
    df_extra["trial_cond"]     = df_extra.apply(get_condition, axis=1)
    df_extra["manip"] =  df_extra.apply(clear_non_conflict_manip, axis=1)

    extra_round_add = generate_numbers()

    df = pd.DataFrame()
    round_count2 = 0
    trial = 1
    for i in range(5):
        for (cond_p1, cond_p2) in conditions:
            
            if trial in extra_round_add:
                df_chunk = df_extra.iloc[:6].copy()
                df_chunk["round_nb"] = trial
                df = pd.concat([df, df_chunk], ignore_index=True)
                df_extra = df_extra.iloc[6:].reset_index(drop=True)
                trial +=1

            for (p1, p2) in rounds[round_count2]:
                aux_df = pd.DataFrame()
                aux_df["participant"] = [p1, p2]
                aux_df["other_participant"] = [p2, p1]
                aux_df["round_nb"] = [trial, trial]
                aux_df["all_cond"] = [cond_p1, cond_p2]
                aux_df["extra_trials"] = False

                df = pd.concat([df, aux_df])
            trial +=1
            round_count2 +=1
            if round_count2 > 4:
                round_count2=0
            

    df["manip"]          = df.apply(get_manip, axis=1)
    df["trial_cond"]     = df.apply(get_condition, axis=1)
    df["manip"] =  df.apply(clear_non_conflict_manip, axis=1)

    
    df.to_csv(f"pairing_{number}.csv")
    number +=1

In [3]:
df = pd.read_csv("test_cleaned.csv")
df["other"] = df.apply(lambda row: row["dyad"].replace("p" + str(row["player"]), ""), axis=1)
df[df["round_nb"] == 1]


,sid,session_code,mk_session,prolific_id,participant_code,round_nb,player,dyad,manipulation,condition,...,rt_individual,indv_stim_order,group_choice,grp_responded_correctly,decision_count,group_rt,both_agree,mean_social_dominance,mean_aggresive_dominance,other
0,interactive_psychophysics,9kiqxmje,interactive_psychophysics,NaN,kr02bapd,1,1,p1p2,submissive,Conflict,...,1984.3,30:40,Right,NaN,2,12381.6,1,NaN,NaN,p2
34,interactive_psychophysics,9kiqxmje,interactive_psychophysics,NaN,9hq8orfh,1,2,p1p2,dominant,Conflict,...,8073.7,40:30,Right,NaN,1,12381.6,1,NaN,NaN,p1
68,interactive_psychophysics,9kiqxmje,interactive_psychophysics,NaN,kiq51pmt,1,3,p3p4,dominant,Conflict,...,158.2,NaN,undecided,NaN,0,171.7,0,NaN,NaN,p4
102,interactive_psychophysics,9kiqxmje,interactive_psychophysics,NaN,vjvuknpg,1,4,p3p4,submissive,Conflict,...,154.6,NaN,undecided,NaN,0,171.7,0,NaN,NaN,p3
136,interactive_psychophysics,9kiqxmje,interactive_psychophysics,NaN,u3eqzh74,1,5,p5p6,dominant,Conflict,...,150.7,NaN,undecided,NaN,0,187.0,0,NaN,NaN,p6
170,interactive_psychophysics,9kiqxmje,interactive_psychophysics,NaN,l2ivemyk,1,6,p5p6,submissive,Conflict,...,153.4,NaN,undecided,NaN,0,187.0,0,NaN,NaN,p5


In [4]:
#df.groupby(["participant"]).count()[["round_nb"]]
df.groupby(["player"]).count()[["round_nb"]]


,round_nb
player,
1,34
2,34
3,34
4,34
5,34
6,34


In [5]:
#df.groupby(["participant", "manip"]).count()[["round_nb"]]
df.groupby(["player", "manipulation"]).count()[["round_nb"]]

round_nb
player manipulation          
1      dominant            17
       submissive          17
2      dominant            17
       submissive          17
3      dominant            17
       submissive          17
4      dominant            17
       submissive          17
5      dominant            17
       submissive          17
6      dominant            17
       submissive          17

In [6]:

#df.groupby(["participant", "other_participant"]).count()[["round_nb"]]
df.groupby(["player", "other"]).count()[["round_nb"]]

round_nb
player other          
1      p2            7
       p3            7
       p4            7
       p5            7
       p6            6
2      p1            7
       p3            6
       p4            7
       p5            7
       p6            7
3      p1            7
       p2            6
       p4            7
       p5            7
       p6            7
4      p1            7
       p2            7
       p3            7
       p5            6
       p6            7
5      p1            7
       p2            7
       p3            7
       p4            6
       p6            7
6      p1            6
       p2            7
       p3            7
       p4            7
       p5            7

In [7]:
df.groupby(["player", "condition"]).count()[["round_nb"]]

round_nb
player condition             
1      Conflict            14
       Non-conflict        20
2      Conflict            14
       Non-conflict        20
3      Conflict            14
       Non-conflict        20
4      Conflict            14
       Non-conflict        20
5      Conflict            14
       Non-conflict        20
6      Conflict            14
       Non-conflict        20

In [8]:

df.groupby(["player", "condition", "difficulty", "manipulation"]).count()[["round_nb"]]

round_nb
player condition    difficulty manipulation          
1      Conflict     Easy       dominant             7
                               submissive           7
       Non-conflict Easy       dominant             5
                               submissive           5
                    Hard       dominant             5
                               submissive           5
2      Conflict     Easy       dominant             7
                               submissive           7
       Non-conflict Easy       dominant             5
                               submissive           5
                    Hard       dominant             5
                               submissive           5
3      Conflict     Easy       dominant             7
                               submissive           7
       Non-conflict Easy       dominant             5
                               submissive           5
                    Hard       dominant             5
                               submissive           5
4      Conflict     Easy       dominant             7
                               submissive           7
       Non-conflict Easy       dominant             5
                               submissive           5
                    Hard       dominant             5
                               submissive           5
5      Conflict     Easy       dominant             7
                               submissive           7
       Non-conflict Easy       dominant             5
                               submissive           5
                    Hard       dominant             5
                               submissive           5
6      Conflict     Easy       dominant             7
                               submissive           7
       Non-conflict Easy       dominant             5
                               submissive           5
                    Hard       dominant             5
                               submissive           5

In [9]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df.groupby(["player", "other", "condition", "difficulty", "manipulation"]).count()[["round_nb"]])

                                                   round_nb
player other condition    difficulty manipulation          
1      p2    Conflict     Easy       dominant             2
                                     submissive           1
             Non-conflict Easy       dominant             1
                                     submissive           1
                          Hard       dominant             1
                                     submissive           1
       p3    Conflict     Easy       dominant             1
                                     submissive           2
             Non-conflict Easy       dominant             1
                                     submissive           1
                          Hard       dominant             1
                                     submissive           1
       p4    Conflict     Easy       dominant             2
                                     submissive           1
             Non-conflict Easy       dom

In [314]:
number = 1
while number < 21:
    #Define rounds
    nc_rounds= [[[1, 2], [4, 3], [6, 5]], #1
            [[1, 3], [5, 2], [4, 6]],   #2
            [[1, 4], [6, 2], [5, 3]],   #3
            [[1, 5], [4, 2], [3, 6]],   #4
            [[1, 6], [3, 2], [4, 5]],   #5
            ]

    #Define different conditions
    nc_conditions = [("ENC_dom", "ENC_sub"),
        ("ENC_sub", "ENC_dom"),
        ("HNC_dom",  "HNC_sub"),
        ("HNC_sub",  "HNC_dom")
    ]

    c_conditions = [ ("EC_dom", "EC_sub"),
        ("EC_sub", "EC_dom")
    ]


    #Shuffle conditions
    random.shuffle(nc_conditions)
    random.shuffle(c_conditions)

    #Complete dataframe with all information
    df = pd.DataFrame()
    cpt = 1

    for (cond1, cond2) in nc_conditions:
        for round in nc_rounds:
            for (p1, p2) in round:
                aux_df = pd.DataFrame()
                aux_df["participant"] = [p1, p2]
                aux_df["other_participant"] = [p2, p1]
                aux_df["round_nb"] = [cpt, cpt]
                aux_df["all_cond"] = [cond1, cond2]
                
                df = pd.concat([df, aux_df])
            cpt+=1

    for (cond1, cond2) in c_conditions:
        
        #Conflict rounds = 7
        c_rounds= [[[1, 2], [4, 3], [6, 5]], #1
                [[1, 3], [5, 2], [4, 6]],   #2
                [[1, 4], [6, 2], [5, 3]],   #3
                [[1, 5], [4, 2], [3, 6]],   #4
                [[1, 6], [3, 2], [4, 5]],   #5
                ]
        #add two more rounds, and randomize pairings. 
        others_round_6 = [2,5,6]
        others_round_7 = [2,5,6]
        random.shuffle(others_round_6)
        random.shuffle(others_round_7)

        round_6 = [[1, others_round_6[0]], [3, others_round_6[1]], [4, others_round_6[2]]]
        round_7 = [[1, others_round_7[0]], [3, others_round_7[1]], [4, others_round_7[2]]]

        c_rounds.append(round_6)  # Add round_6 to the list
        c_rounds.append(round_7)  # Add round_7 to the list

        for round in c_rounds:
            for (p1, p2) in round:
                aux_df = pd.DataFrame()
                aux_df["participant"] = [p1, p2]
                aux_df["other_participant"] = [p2, p1]
                aux_df["round_nb"] = [cpt, cpt]
                aux_df["all_cond"] = [cond1, cond2]
                
                df = pd.concat([df, aux_df])
            cpt+=1
            

    #Add useful information
    df["manip"]          = df.apply(get_manip, axis=1)
    df["trial_cond"]     = df.apply(get_condition, axis=1)
    df["manip"] =  df.apply(clear_non_conflict_manip, axis=1)

    #Randomize round numbers
    unique_rounds = df['round_nb'].unique()

    # Shuffle the unique round numbers
    shuffled_rounds = np.random.permutation(unique_rounds)

    # Create a mapping from original rounds to shuffled rounds
    round_mapping = dict(zip(unique_rounds, shuffled_rounds))

    # Apply the mapping to the dataframe
    df['round_nb'] = df['round_nb'].map(round_mapping)

    df = df.sort_values("round_nb")


    players = ["1", "2", "3", "4", "5", "6"]
    df["other_player_str"] = df["other_participant"].astype(str)
    df["participant_str"] = df["participant"].astype(str)

    distance_dict = {}

    for player in players:
        df_player     = df[df["participant_str"] == player]
        other_players = [p for p in players if p != player]

        distance_dict[player] = 0
        for other in other_players:
            
            last_seen = 0
            for index,row in df_player.iterrows():
                if other == row["other_player_str"]:
                    distance  = row["round_nb"] - last_seen
                    last_seen = row["round_nb"]
                    if distance == 1:
                        distance_dict[f"{player}"] += 1
                    
    distance_dict

    if distance_dict["1"] < 3:
        print(distance_dict)
        print(f"found distance less than 3, saving csv number {number}")
        df.to_csv(f"pairing_{number}.csv")

        number += 1

{'1': 2, '2': 2, '3': 4, '4': 3, '5': 4, '6': 3}
found distance less than 4, saving csv number 1
{'1': 2, '2': 3, '3': 2, '4': 4, '5': 2, '6': 3}
found distance less than 4, saving csv number 2
{'1': 2, '2': 2, '3': 4, '4': 5, '5': 6, '6': 3}
found distance less than 4, saving csv number 3
{'1': 2, '2': 4, '3': 3, '4': 5, '5': 3, '6': 3}
found distance less than 4, saving csv number 4
{'1': 2, '2': 3, '3': 3, '4': 2, '5': 2, '6': 2}
found distance less than 4, saving csv number 5
{'1': 2, '2': 2, '3': 4, '4': 2, '5': 4, '6': 2}
found distance less than 4, saving csv number 6
{'1': 2, '2': 2, '3': 5, '4': 3, '5': 5, '6': 3}
found distance less than 4, saving csv number 7
{'1': 1, '2': 1, '3': 3, '4': 2, '5': 3, '6': 2}
found distance less than 4, saving csv number 8
{'1': 2, '2': 2, '3': 2, '4': 3, '5': 3, '6': 2}
found distance less than 4, saving csv number 9
{'1': 1, '2': 1, '3': 3, '4': 3, '5': 4, '6': 2}
found distance less than 4, saving csv number 10
{'1': 2, '2': 3, '3': 3, '4':

In [20]:
#Define rounds
rounds=  [[[1, 2], [4, 3], [6, 5]], #1
        [[1, 3], [5, 2], [4, 6]],   #2
        [[1, 4], [6, 2], [5, 3]],   #3
        [[1, 5], [4, 2], [3, 6]],   #4
        [[1, 6], [3, 2], [4, 5]],   #5
        ]

#Define different conditions
conditions = [ ("EC_dom", "EC_sub"),
    ("EC_sub", "EC_dom"), 
    ("ENC_dom", "ENC_sub"),
    ("ENC_sub", "ENC_dom"),
    ("HNC_dom",  "HNC_sub"),
    ("HNC_sub",  "HNC_dom")
]
number = 1
while number <= 1:

    # Initialize the restart flag
    restart = False

    #Shuffle conditions
    random.shuffle(conditions)
    print(conditions)

    #Complete dataframe with all information
    df = pd.DataFrame()
    cpt = 1
    for (cond1, cond2) in conditions:
        for round in rounds:
            for (p1, p2) in round:
                aux_df = pd.DataFrame()
                aux_df["participant"] = [p1, p2]
                aux_df["other_participant"] = [p2, p1]
                aux_df["round_nb"] = [cpt, cpt]
                aux_df["all_cond"] = [cond1, cond2]
                
                df = pd.concat([df, aux_df])
            cpt+=1
        

    #Add useful information
    df["manip"]          = df.apply(get_manip, axis=1)
    df["trial_cond"]     = df.apply(get_condition, axis=1)
    df["manip"] =  df.apply(clear_non_conflict_manip, axis=1)

    #Randomize round numbers
    unique_rounds = df['round_nb'].unique()

    # Shuffle the unique round numbers
    shuffled_rounds = np.random.permutation(unique_rounds)

    # Create a mapping from original rounds to shuffled rounds
    round_mapping = dict(zip(unique_rounds, shuffled_rounds))

    # Apply the mapping to the dataframe
    df['round_nb'] = df['round_nb'].map(round_mapping)

    df = df.sort_values("round_nb")
       
    number +=1  


#df.to_csv(f"pairing_{number}.csv")
#number+=1

players = ["1", "2", "3", "4", "5", "6"]
df["other_player_str"] = df["other_participant"].astype(str)
df["participant_str"] = df["participant"].astype(str)

distance_dict = {}

for player in players:
    df_player     = df[df["participant_str"] == player]
    other_players = [p for p in players if p != player]

    distance_dict[player] = 0
    for other in other_players:
        
        last_seen = 0
        for index,row in df_player.iterrows():
            if other == row["other_player_str"]:
                distance  = row["round_nb"] - last_seen
                last_seen = row["round_nb"]
                if distance == 1:
                    distance_dict[f"{player}"] += 1
                
distance_dict

#6 times it happens that participants are paired consecutively in the first pilot. 
#the minimum number of times it can happen is 2 per participant. We want to collect these csvs. 

[('ENC_dom', 'ENC_sub'), ('ENC_sub', 'ENC_dom'), ('HNC_dom', 'HNC_sub'), ('EC_dom', 'EC_sub'), ('HNC_sub', 'HNC_dom'), ('EC_sub', 'EC_dom')]


{'1': 2, '2': 2, '3': 2, '4': 2, '5': 2, '6': 2}